качаем Glove, хороший список моделей тут:

https://github.com/3Top/word2vec-api

In [1]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2017-02-19 21:25:07--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

100%[======================================>] 862 182 613  617KB/s   in 27m 46s

2017-02-19 21:52:53 (505 KB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [2]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


используем convert.py из  https://raw.githubusercontent.com/jroakes/glove-to-word2vec/master/convert.py
По умолчанию он конвертирует модель размерности 300, мы попробуем размерность 50

In [2]:
!python convert.py

OpenBLAS : Your OS does not support AVX instructions. OpenBLAS is using Nehalem kernels as a fallback, which may give poorer performance.
OpenBLAS : Your OS does not support AVX instructions. OpenBLAS is using Nehalem kernels as a fallback, which may give poorer performance.
/usr/local/lib/python2.7/dist-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")
[(u'zealand', 0.9042125940322876), (u'england', 0.7858781814575195), (u'australian', 0.7696145176887512), (u'britain', 0.7670673131942749), (u'canada', 0.7569618821144104), (u'africa', 0.7522280216217041), (u'scotland', 0.7264115214347839), (u'wales', 0.720910906791687), (u'india', 0.704209566116333), (u'indies', 0.6983797550201416)]
0.832349418276


In [1]:
import gensim
w2v = gensim.models.Word2Vec.load_word2vec_format('./glove_model2.txt', binary=False)

/usr/local/lib/python2.7/dist-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


используем центроид вместо неизвестных слов

In [2]:
import numpy  as np
centroid = np.mean(w2v.syn0, axis=0)
print centroid.shape

(50,)


Качаем архив из http://alt.qcri.org/semeval2015/task2/data/uploads/sts2015-en-post.zip   

Создаем сразу выборки как для RNN, так и для baseline в статье (нужно убедиться что мы все напрогали правильно и RNN работает лучше базовых алгоритмов)

In [4]:
from collections import Counter
def cosine_tokens(tokens1, tokens2):
    c1 = Counter(tokens1)
    c2 = Counter(tokens2)
    intersection = set(c1.keys()) & set(c2.keys())
    numerator = sum([c1[x] * c2[x] for x in intersection])
    sum1 = sum([c1[x]**2 for x in c1.keys()])
    sum2 = sum([c2[x]**2 for x in c2.keys()])
    denominator = np.sqrt(sum1) * np.sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator
    

In [5]:
def jaccard(tokens1, tokens2):
    set1 = set(tokens1)
    set2 = set(tokens2)
    return float(len(set1 | set2))/(len(set1) + len(set2) - len(set1&set2))

In [10]:
from scipy.spatial.distance import cosine
filename = 'text.clean'
X,Y = [],[]
X_base = []
with open(filename) as inp:    
    for line in inp:    
        sublines = line.split('\t')
        score = int(np.round(float(sublines[0])).astype(np.uint))
        vectors = []
        tokens = []
        for subline in [sublines[4],sublines[5]]:
            subline = re.sub('[^a-z\s]+', ' ', subline.lower())
            tokens.append([t for t in subline.split() if t not in stopwords])
            vectors.append([w2v[t] if t in w2v.vocab else centroid for t in tokens[-1] ])
        if len(vectors[0])==0 :
            vectors[0].append(centroid)
        if len(vectors[1])==0:
            vectors[1].append(centroid)
        X.append(vectors[0])
        X.append(vectors[1])
        X_base.append([cosine(np.mean(vectors[0], axis=0),np.mean(vectors[1],axis=0)), cosine_tokens(tokens[0], tokens[1]),
                      jaccard(tokens[0], tokens[1])])
        Y.append(score)
idx = range(len(Y))
np.random.shuffle(idx)

X_base = [X_base[i] for i in idx]
Y = [Y[i] for i in idx]
X_new = []
for i in idx:
    X_new.append(X[2*i])
    X_new.append(X[2*i+1])
np.save('semeval_x', X_new)
np.save('semeval_x_baseline', X_base)
np.save('semeval_y', Y)